In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 20.1MB/s]


In [10]:
import os
import json

def coco_to_yolo(coco_json, img_dir, output_dir):
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)

    # Creare la directory per le annotazioni se non esiste
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Mappa classi COCO a classi YOLO
    category_mapping = {category['id']: idx for idx, category in enumerate(coco_data['categories'])}

    # Per ogni immagine nel dataset
    for image_info in coco_data['images']:
        img_id = image_info['id']
        img_width = image_info['width']
        img_height = image_info['height']
        img_filename = os.path.splitext(image_info['file_name'])[0]

        # Crea un file .txt con lo stesso nome dell'immagine
        output_file = os.path.join(output_dir, f"{img_filename}.txt")
        
        with open(output_file, 'w') as f_txt:
            # Trova tutte le annotazioni relative a questa immagine
            for ann in coco_data['annotations']:
                if ann['image_id'] == img_id:
                    # Prendi le coordinate del bounding box (COCO usa [x_min, y_min, width, height])
                    x_min, y_min, width, height = ann['bbox']

                    # Converti in formato YOLO [x_center, y_center, width, height] e normalizza
                    x_center = x_min + width / 2
                    y_center = y_min + height / 2

                    x_center /= img_width
                    y_center /= img_height
                    width /= img_width
                    height /= img_height

                    # Clipping per garantire che i valori siano tra 0 e 1
                    x_center = min(max(x_center, 0), 1)
                    y_center = min(max(y_center, 0), 1)
                    width = min(max(width, 0), 1)
                    height = min(max(height, 0), 1)

                    # Ottieni il category_id e lo mappa al formato YOLO
                    category_id = category_mapping[ann['category_id']]

                    # Scrivi la linea nel file di annotazione YOLO
                    f_txt.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

    print(f"Conversione completata! Le annotazioni YOLO sono state salvate in: {output_dir}")


In [11]:
coco_json = "human/train/annotation_train.json"  # File JSON COCO
img_dir = "human/train"  # Directory delle immagini
output_dir = "human/train"  # Directory dove salvare le annotazioni YOLO

coco_to_yolo(coco_json, img_dir, output_dir)


coco_json = "human/validation/annotation_val.json"  # File JSON COCO
img_dir = "human/validation"  # Directory delle immagini
output_dir = "human/validation"  # Directory dove salvare le annotazioni YOLO

coco_to_yolo(coco_json, img_dir, output_dir)


Conversione completata! Le annotazioni YOLO sono state salvate in: human/train
Conversione completata! Le annotazioni YOLO sono state salvate in: human/validation


In [18]:
# Train the model
train_results = model.train(
    data="human/human_dataset.yaml",  # path to dataset YAML
    epochs=4,  # number of training epochs
    imgsz=32,  # training image size, piu grande e' meglio e' per oggetti piccoli, solo multipli di 32
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

Ultralytics 8.3.18 🚀 Python-3.10.13 torch-2.5.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=human/human_dataset.yaml, epochs=4, time=None, patience=100, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=cpu, workers=0, project=None, name=train73, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None

train: Scanning /Users/stefano_ruggiero/Desktop/3D_perception_progetto/human/train.cache... 15000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15000/15000 [00:00<?, ?it/s]
val: Scanning /Users/stefano_ruggiero/Desktop/3D_perception_progetto/human/validation.cache... 4370 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4370/4370 [00:00<?, ?it/s]


Plotting labels to runs/detect/train73/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 32 train, 32 val
Using 0 dataloader workers
Logging results to runs/detect/train73
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/4         0G      3.874      1.364      1.189        191         32: 100%|██████████| 938/938 [15:45<00:00,  1.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 137/137 [02:04<00:00,  1.10it/s]

                   all       4370     127716      0.188     0.0532     0.0735     0.0217


...

        4/4         0G      3.285      1.206      1.033        133         32: 100%|██████████| 938/938 [16:18<00:00,  1.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 137/137 [02:01<00:00,  1.12it/s]

                   all       4370     127716      0.208     0.0588     0.0503     0.0152



4 epochs completed in 1.228 hours.
Optimizer stripped from runs/detect/train73/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train73/weights/best.pt, 5.4MB

Validating runs/detect/train73/weights/best.pt...
Ultralytics 8.3.18 🚀 Python-3.10.13 torch-2.5.0 CPU (Apple M1)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 137/137 [01:57<00:00,  1.16it/s]


                   all       4370     127716      0.188     0.0533     0.0736     0.0217
Speed: 0.0ms preprocess, 10.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train73


In [21]:
# Evaluate model performance on the validation set
metrics = model.val()

Ultralytics 8.3.18 🚀 Python-3.10.13 torch-2.5.0 CPU (Apple M1)


val: Scanning /Users/stefano_ruggiero/Desktop/3D_perception_progetto/human/validation.cache... 4370 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4370/4370 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 274/274 [02:11<00:00,  2.09it/s]


                   all       4370     127716      0.189     0.0533     0.0736     0.0217
Speed: 0.0ms preprocess, 13.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train734


In [28]:
#print(metrics)

print("Overall AP:", metrics.box.map)  # mAP averaged across IoU thresholds
print("AP50:", metrics.box.map50)      # mAP at IoU=0.50
print("AP75:", metrics.box.map75)      # mAP at IoU=0.75
print("AP for small objects:", metrics.box.maps)  # mAP for small objects



# Perform object detection on an example image
results = model("human/validation/273271,1b9330008da38cd6.jpg")
results[0].show()

Overall AP: 0.021709310622931813
AP50: 0.07361445171058154
AP75: 0.007511080194068893
AP for small objects: [   0.021709]

image 1/1 /Users/stefano_ruggiero/Desktop/3D_perception_progetto/human/validation/273271,1b9330008da38cd6.jpg: 32x32 10 persons, 29.9ms
Speed: 2.1ms preprocess, 29.9ms inference, 1.5ms postprocess per image at shape (1, 3, 32, 32)


python(39266) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(39267) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [29]:
# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

Ultralytics 8.3.18 🚀 Python-3.10.13 torch-2.5.0 CPU (Apple M1)

PyTorch: starting from 'runs/detect/train73/weights/best.pt' with input shape (1, 3, 32, 32) BCHW and output shape(s) (1, 5, 21) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim'] not found, attempting AutoUpdate...


python(39318) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.0 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 6.5s, installed 2 packages: ['onnx>=1.12.0', 'onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.35...
ONNX: export success ✅ 14.1s, saved as 'runs/detect/train73/weights/best.onnx' (9.9 MB)

Export complete (14.2s)
Results saved to /Users/stefano_ruggiero/Desktop/3D_perception_progetto/runs/detect/train73/weights
Predict:         yolo predict task=detect model=runs/detect/train73/weights/best.onnx imgsz=32  
Validate:        yolo val task=detect model=runs/detect/train73/weights/best.onnx imgsz=32 data=human/human_dataset.yaml  
Visualize:       https://netron.app
